In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

# Load the data
master = pd.read_csv('master1.csv')

# Filter relevant crime types
relevant_crime_types = ['HOMICIDE', 'BATTERY', 'ASSAULT', 'ROBBERY', 'CRIMINAL SEXUAL ASSAULT']
master = master[master['Primary Type'].isin(relevant_crime_types)]

# Drop irrelevant columns
columns_to_drop = ['Case Number', 'Time', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic','Beat','District','Ward','Community Area','FBI Code','X Coordinate',
                'Y Coordinate','Updated On','Latitude','Longitude','Location', 'Holiday Day of Week','precipprob','snowdepth','preciptype', 'windgust','winddir',
                'solarenergy','sunrise', 'sunset','moonphase', 'description', 'icon','stations']
data = master.drop(columns=columns_to_drop)

# Replace NaN values with 0 indicating no holiday and severe risk
data['Holiday'] = data['Holiday'].fillna(0)
data['severerisk'] = data['severerisk'].fillna(0)

# Replace non-NaN values with 1 indicating a holiday
data['Holiday'] = data['Holiday'].apply(lambda x: 1 if x != 0 else 0)

# Filter data for years after 2010
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%y')
data = data[data['Date'].dt.year >= 2010]

# Group by date and get crime counts
daily_counts = data.groupby('Date').size().reset_index(name='Crime_Count')

# Merge daily counts with the original DataFrame
merged_data = pd.merge(data, daily_counts, on='Date')

# Drop duplicate rows to keep only one entry per day
final_data = merged_data.drop_duplicates(subset='Date')

# Split conditions column and create dummy variables
final_data['conditions'] = final_data['conditions'].str.split(',')
final_data = final_data.join(final_data['conditions'].str.join('|').str.get_dummies())

# Drop unnecessary columns
final_data.drop(columns=['conditions'], inplace=True)

final_data['Crime_Count'] = np.sqrt(final_data['Crime_Count'])


# Split the data into features (X) and target variable (y)
X = final_data.drop(columns=['Crime_Count', 'ID', 'Date'])
y = final_data['Crime_Count']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestRegressor


# Train the Random Forest model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

# Predict the test set results
y_pred = rf_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

C:\Users\Matt\AppData\Local\Temp\ipykernel_11640\2966156428.py:10: DtypeWarning: Columns (23,24,36,49,50,52,53,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  master = pd.read_csv('master1.csv')
C:\Users\Matt\AppData\Local\Temp\ipykernel_11640\2966156428.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['conditions'] = final_data['conditions'].str.split(',')


Mean Squared Error: 0.3504084294635533
R^2 Score: 0.24238923731025663
